<a href="https://colab.research.google.com/github/FatmaBuseBorlu/YuzGoruntulerindenDuyguTanimaveTepkiVerme/blob/main/Yapay_Zeka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [2]:
from google.colab import files
import zipfile
import os

# Yerel bilgisayardan dosya yükleme
uploaded = files.upload()

# Yüklenen dosyayı kontrol etme
for file_name in uploaded.keys():
    print(f"Yüklendi: {file_name}")

    # Eğer dosya bir ZIP ise, çıkar
    if file_name.endswith(".zip"):
        with zipfile.ZipFile(file_name, 'r') as zip_ref:
            zip_ref.extractall()  # Varsayılan dizine çıkarır
            print(f"{file_name} başarıyla çıkarıldı.")

Saving RAF-DB-DATASET.zip to RAF-DB-DATASET.zip
Yüklendi: RAF-DB-DATASET.zip
RAF-DB-DATASET.zip başarıyla çıkarıldı.


In [3]:
# Veri yükleme fonksiyonu
def load_data(directory):
    X, y = [], []
    for label in os.listdir(directory):
        label_path = os.path.join(directory, label)
        if os.path.isdir(label_path):
            for image_name in os.listdir(label_path):
                image_path = os.path.join(label_path, image_name)
                img = load_img(image_path, target_size=(224, 224))
                img_array = img_to_array(img)
                img_array = preprocess_input(img_array)
                X.append(img_array)
                y.append(int(label))
    return np.array(X), np.array(y)

# Eğitim ve test veri yolları
train_dir = "/content/RAF-DB-DATASET/DATASET/train"
test_dir = "/content/RAF-DB-DATASET/DATASET/test"

# Eğitim ve test verilerini yükleme
X_train, y_train = load_data(train_dir)
X_test, y_test = load_data(test_dir)

print("Veriler başarıyla yüklendi.")

Veriler başarıyla yüklendi.


In [4]:
# SMOTE ile veri artırma
from imblearn.over_sampling import SMOTE

# SMOTE uygulamadan önce veriyi yeniden şekillendir
X_train_flat = X_train.reshape(X_train.shape[0], -1)

# SMOTE uygulama
smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_resample(X_train_flat, y_train)

# SMOTE sonrası veriyi yeniden şekillendir
X_train = X_train_smote.reshape(X_train_smote.shape[0], 224, 224, 3)
y_train = y_train_smote

# Boyutları kontrol et
print(f"X_train boyutu: {X_train.shape}")
print(f"y_train boyutu: {y_train.shape}")


X_train boyutu: (33404, 224, 224, 3)
y_train boyutu: (33404,)


In [5]:
# Veri artırma işlemi
data_augmentation = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [6]:
print(f"X_train boyutu: {X_train.shape}")
print(f"y_train boyutu: {y_train.shape}")


X_train boyutu: (33404, 224, 224, 3)
y_train boyutu: (33404,)


In [7]:
# EfficientNetB0 tabanlı model oluşturma
num_classes = 7 # Sınıf sayısı (Surprise, Fear, Disgust, Happy, Sad, Anger, Neutral)

base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.6),  # %60 Dropout
    Dense(512, activation='relu'),
    Dropout(0.6),  # %60 Dropout
    Dense(num_classes, activation='softmax')
])


# Adam optimizer ve Learning Rate Scheduler
initial_learning_rate = 0.001
def lr_scheduler(epoch, lr):
    if epoch > 10:
        return lr * 0.5
    return lr

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [8]:
# Model derleme
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Daha düşük öğrenme oranı
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [9]:
# Etiketlerin maksimum değerini kontrol et
print(f"Etiketlerin maksimum değeri (y_train): {y_train.max()}")
print(f"Etiketlerin maksimum değeri (y_test): {y_test.max()}")

# Etiketleri sıfırdan başlatma ve yeniden etiketleme
y_train -= y_train.min()
y_test -= y_test.min()

# Etiketlerin kontrolünü tekrar yap
print(f"Düzenlenmiş etiketlerin maksimum değeri (y_train): {y_train.max()}")
print(f"Düzenlenmiş etiketlerin maksimum değeri (y_test): {y_test.max()}")


Etiketlerin maksimum değeri (y_train): 7
Etiketlerin maksimum değeri (y_test): 7
Düzenlenmiş etiketlerin maksimum değeri (y_train): 6
Düzenlenmiş etiketlerin maksimum değeri (y_test): 6


In [10]:
# Callback'ler
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_reduction = LearningRateScheduler(lr_scheduler)

In [ ]:
history = model.fit(
    data_augmentation.flow(X_train, y_train, batch_size=32),
    epochs=50,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, lr_reduction]
)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1044/1044 ━━━━━━━━━━━━━━━━━━━━ 403s 335ms/step - accuracy: 0.3123 - loss: 1.7352 - val_accuracy: 0.6519 - val_loss: 0.9773 - learning_rate: 1.0000e-04
Epoch 2/50
1044/1044 ━━━━━━━━━━━━━━━━━━━━ 310s 294ms/step - accuracy: 0.6571 - loss: 0.9539 - val_accuracy: 0.7151 - val_loss: 0.8209 - learning_rate: 1.0000e-04
Epoch 3/50
1044/1044 ━━━━━━━━━━━━━━━━━━━━ 312s 297ms/step - accuracy: 0.7441 - loss: 0.7146 - val_accuracy: 0.7412 - val_loss: 0.7525 - learning_rate: 1.0000e-04
Epoch 4/50
 558/1044 ━━━━━━━━━━━━━━━━━━━━ 2:21 292ms/step - accuracy: 0.7927 - loss: 0.5836

In [ ]:
# Modeli test etme
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Doğruluğu: {test_accuracy * 100:.2f}%")

In [ ]:
# Modelin tahminlerini değerlendirme
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# y_test is already a 1D array of true labels, no need for argmax
y_true_classes = y_test

In [ ]:
# Hata Matrisi ve Sınıflandırma Raporu
cm = confusion_matrix(y_true_classes, y_pred_classes)
print("Confusion Matrix:\n", cm)
print("Classification Report:\n", classification_report(y_true_classes, y_pred_classes))

In [ ]:
# Confusion matrix görselleştirme
import seaborn as sns
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=range(num_classes), yticklabels=range(num_classes))
plt.xlabel("Tahmin Edilen")
plt.ylabel("Gerçek")
plt.show()

In [ ]:
# Eğitim kaybı ve doğruluk grafiği çizme
def plot_learning_curves(history):
    plt.figure(figsize=(12, 4))
    plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
    plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
    plt.title("Eğitim ve Doğrulama Doğruluğu")
    plt.xlabel("Epoch")
    plt.ylabel("Doğruluk")
    plt.legend()
    plt.show()

plot_learning_curves(history)

In [ ]:
# Modeli kaydetme
model.save("emotion_recognition_model.h5")
print("Model başarıyla kaydedildi.")


In [ ]:
# Modeli yükleme
model_path = "/content/emotion_recognition_model.h5"
model = load_model(model_path)

# Test CSV dosyasını yükleme
test_csv_path = "/content/drive/MyDrive/Yapay Zeka Bilişsel Sİstemler Dersi Proje/RAF-DB-DATASET/test_labels.csv"
test_data = pd.read_csv(test_csv_path)

# Rastgele bir görüntü seçme
random_row = test_data.sample(n=1).iloc[0]
test_image_name = random_row['image']
correct_label = random_row['label']

# Görüntü yolunu oluşturma
test_image_path = os.path.join("/content/RAF-DB-DATASET/DATASET/test", str(correct_label), test_image_name)
print(f"Test edilecek görüntü: {test_image_path}")

# Görüntüyü yükleme ve işleme
img = load_img(test_image_path, target_size=(224, 224))  # Görüntüyü yükle ve boyutlandır
img_array = img_to_array(img) / 255.0  # Normalizasyon
img_array = np.expand_dims(img_array, axis=0)  # Modelin tahmini için batch boyutuna dönüştür

# Görüntü üzerinde tahmin yapma
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)

# Sınıf etiketleri
emotion_labels = {0: "Surprise", 1: "Fear", 2: "Disgust", 3: "Happy", 4: "Sad", 5: "Anger", 6: "Neutral"}
predicted_label = emotion_labels[predicted_class]

# Tahminin görselleştirilmesi
plt.imshow(load_img(test_image_path))  # Görüntüyü yükleyerek görselleştir
plt.title(f"Tahmin: {predicted_label}")
plt.axis("off")
plt.show()

print(f"Modelin tahmini: {predicted_label}")